# Fourier transforms

If a function is defined on the whole real line and not just a single interval (although it may have finite support), it is not necessarily period. It can have sine-like components at all frequencies (wavenumbers), not just those that are interval multiples of $2\pi/L$. The 'coefficients' of such a 'continuous series' are the Fourier transform. 

In general, the Fourier transform of a function is complex even if the function is real. However, if the function is even about $x = 0$ then its FT is real; if it is odd, then its FT is pure imaginary.

Computers can't represent analogue functions exactly. They can only sample them at discrete points. The Fourier transform of a discretely sampled function is called the discrete Fourier transform (DFT). This means that there is a limit to the maximum frequency that can be calculated by the DFT. There is also a limit to the domain width that can be stored, so there is also a minimum frequency of the DFT.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def Vectorize(f):
    """Wrapper around np.vectorize for floats"""
    return np.vectorize(f, otypes=[float])

def myfft(ys):
    """Our FT convention that we will use in this course.
    Note that conventions vary between packages and authors
    over the normalisation.
    For this course, we want our FTs to be independent of
    the sample rate, so we'll normalise by multiplying by 2/nx.
    """
    # TODO - sort out the normalisation. It's wrong.
    return np.fft.fft(ys, norm="ortho") * 2 / (np.pi * len(ys))

In [ ]:
# The domain
def get_domain(nx, xmax):
    xs = np.linspace(-xmax, xmax, nx+1)[:-1]
    dx = xs[1] - xs[0]
    nyquist = np.pi / dx
    ks = np.linspace(-nyquist, nyquist, nx+1)[:-1]
    dk = ks[1] - ks[0]
    return [xs, dx, ks, dk]

## Question 7: FTs of functions of finite extent

In [ ]:
def CompactSupport(xmin, xmax):
    def compact_f(f):
        return lambda x: 0 if (x > xmax or x < xmin) else f(x)

    return compact_f

@interact(c=widgets.FloatSlider(min=0, max=3, value=0.4),
          a=widgets.FloatSlider(min=0, max=10, value=1),
          fn=["tophat", "sinax", "cosax"])
def q7demo(c, a, fn):
    @Vectorize
    @CompactSupport(-c, c)
    def tophat(x):
        return 1
    
    @Vectorize
    @CompactSupport(-c, c)
    def sinax(x):
        return np.sin(a*x)
    
    @Vectorize
    @CompactSupport(-c, c)
    def cosax(x):
        return np.cos(a*x)
    
    xs, dx, ks, dk = get_domain(1024, xmax=24)
    ys = locals()[fn](xs)
    yfts = np.fft.fftshift(myfft(ys))
#     print(np.sum(np.real(yfts)) * dk)
#     print(np.sum(np.abs(ys)**2) * dx, np.sum(np.abs(yfts) ** 2 * dk))
    fig, axs = plt.subplots(1, 2, figsize=[14, 5])
    axs[0].plot(xs, ys, 'k-')
    axs[0].set(xlim=[-4,4])
    axs[1].plot(ks, np.real(yfts), 'k-',
                ks, np.imag(yfts), 'b--')
    axs[1].set(xlim=[-15,15])
    plt.show()

## Question 9: FTs of Gaussians

In [ ]:
@interact_manual(n=widgets.FloatSlider(min=0, max=8, value=1),
          m=widgets.FloatSlider(min=-5, max=5, value=0))
def q8demo(n, m):
    @Vectorize
    def gaussian(x):
        return np.exp(-n**2 * (x - m)**2)
    
    xs, dx, ks, dk = get_domain(nx=1024, xmax=32)
    ys = gaussian(xs)
    yfts = np.fft.fftshift(myfft(ys))
#     print(np.sum(np.real(yfts)) * dk)
#     print(np.sum(np.abs(ys)**2) * dx, np.sum(np.abs(yfts) ** 2 * dk))
    fig, axs = plt.subplots(2, 2, figsize=[14, 8])
    axs[0, 0].plot(xs, ys, 'k-')
    axs[0, 0].set(xlim=[-10,10])
    axs[0, 1].plot(ks, np.abs(yfts), 'k-')
    axs[0, 1].set(xlim=[-15,15])
    axs[1, 0].plot(ks, np.real(yfts), 'k-')
    axs[1, 1].plot(ks, np.imag(yfts), 'b-')
    axs[1, 0].set(xlim=[-15,15])
    axs[1, 1].set(xlim=[-15,15])
    plt.show()